In [1]:
import os
import numpy as np
from numba import jit
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Configuration
doc_dir = 'Document/'
query_dir = 'Query/'

# Hyperparameters
min_df = 1         # Exclude words with document frequency < min_df for unigram and PLSA
topic_num = 30     # Topic numbers of PLSA
max_iter = 1000     # Max iterations of PLSA
threshold = 1e-4   # Decide convergence of PLSA for early stopping (Refer to PLSA() below for details)

alpha = 0.1   # Weight of unigram model
beta = 0.7    # Weight of PLSA

In [3]:
%%time
# Read 'Collections.txt' as train set
train_set = []
with open('Collection.txt', mode='r') as file:
    train_set += [line.rstrip() for line in file]

# Read documents as test set
test_set = []
doc_list = os.listdir(doc_dir)
for doc_name in doc_list:
    doc_path = doc_dir + doc_name
    with open(doc_path, mode="r") as file:
        lines = file.readlines()[3:]   # First three lines are useless headers
        lines = [line.rstrip()[:-3] for line in lines]   # Strip tailing -1 from each line
        document = ' '.join(lines)
        test_set.append(document)

CPU times: user 168 ms, sys: 60 ms, total: 228 ms
Wall time: 206 ms


In [4]:
%%time
# Process train set into doc-term matrix
vectorizer = CountVectorizer(token_pattern='[0-9]+', min_df=min_df)
doc_term = vectorizer.fit_transform(train_set).tocoo()  # i.e. c(w,d)
doc_count, vocab_size = doc_term.shape
vocab_table = vectorizer.vocabulary_  # Mapping of {word -> col of doc_term}

# Process test set into doc-term matrix for fold-in usage
doc_term_fi = vectorizer.transform(test_set).tocoo()
doc_count_fi = doc_term_fi.shape[0]

CPU times: user 3.49 s, sys: 64 ms, total: 3.55 s
Wall time: 3.55 s


In [5]:
@jit(nopython=True)
def plsa(doc_term_data, doc_term_row, doc_term_col, doc_topic, topic_word,
         fold_in=False, max_iter=max_iter, threshold=threshold, verbose=1):
    ''' PLSA implemented with sparse matrix and JIT compilation.
    
    Arguments are passed by references to functions defined with numba JIT,
    thus you should manually initialize PLSA parameters and pass them in.
    
    Also, data, row, col attributes of COO matrix are directly passed in
    since numba JIT does not recognize any type of scipy sparse matrix.
    
    
    Args:
        doc_term_data: Non-zero entry of doc-term matrix.
        doc_term_row : Row index of entires in doc_term_data.
        doc_term_col : Column index of entries in doc_term_data.
        
        doc_topic    : Numpy array in shape of (n_doc, n_topic).  i.e. P(T|d)
        topic_word   : Numpy array in shape of (n_topic, n_word). i.e. P(w|T)
        
        fold_in      : Whether to perform fold-in strategy.
        max_iter     : Max iterations of EM algorithm.
        threshold    : Decide convergence of model for early stopping.
            Training stops as (loglike - prev loglike) / |prev loglike| < threshold.
            
        verbose      : Control the frequency to print log-likelihood.
                        <1: Print nothing.
                         0: Print only at the end of training.
                        >0: Print for every [verbose] iterations.
    '''

    ####################
    #  Initialization
    ####################
    doc_count, topic_num = doc_topic.shape
    vocab_size = topic_word.shape[1]
    nnz = len(doc_term_data)  # Number of non-zero entries in doc-term matrix
    
    docword_topic = np.zeros((nnz, topic_num))  # P(T|w,d)  i.e E-step要算的東西
    doc_topic_sum = np.zeros((doc_count))       # 更新P(T|d)時的分母項
    if not fold_in:
        topic_word_sum = np.zeros((topic_num))  # 更新P(w|T)時的分母項
    
    
    prev_log_like = 0
    for it in range(1, max_iter+1):
        log_like = 0
        ############
        #  E-step
        ############
        for dwi in range(nnz):  # 更新P(T|w,d)時，對每一組非零之(w,d)逐個更新
            di, wi = doc_term_row[dwi], doc_term_col[dwi]
            joint_prob = np.zeros((topic_num))  # 分子項
            joint_prob_sum = 0                  # 分母項
            
            for ti in range(topic_num): # P(T|w,d)的 T值也要逐個更新
                joint_prob[ti] = doc_topic[di, ti] * topic_word[ti, wi] # P(w|T)P(T|d)
                joint_prob_sum += joint_prob[ti]  # 把 P(w|T)P(T|d) 沿著 T 加總
                
            # log-likelihood (把目標函式擺在這計算比較有效率，不過也是要對每組(w,d)逐個加總)
            log_like += doc_term_data[dwi] * np.log(joint_prob_sum)
            
            # Normalization  (i.e. 就是分子除以分母啦)
            for ti in range(topic_num):
                docword_topic[dwi, ti] = joint_prob[ti] / joint_prob_sum
                
                
        # Early stopping
        if prev_log_like != 0 and (log_like - prev_log_like) / abs(prev_log_like) < threshold:
            if verbose > -1:
                print("--- Early stopping at iteration", it, "---")
                print("loglike:", log_like)
                return
            
        if verbose > 0 and it % verbose == 0:
            print("iter", it, "- loglike:", log_like)
        prev_log_like = log_like

        
        ##################################################
        #  M-step  (如果你E-step看懂了，M-step一定也看得懂)
        ##################################################
        # 要更新的東西需要先全部歸零
        doc_topic.fill(0)
        doc_topic_sum.fill(0)
        if not fold_in:
            topic_word.fill(0)
            topic_word_sum.fill(0)

        for dwi in range(nnz):
            di, wi = doc_term_row[dwi], doc_term_col[dwi]
            for ti in range(topic_num):
                likelihood = doc_term_data[dwi] * docword_topic[dwi, ti]  # c(w,d)P(T|w,d)
                doc_topic[di, ti] += likelihood   # 因為index只用了di,ti，所以likelihood會沿著w加總
                doc_topic_sum[di] += likelihood   # 同理，這次連ti都拔了，所以likelihood會沿著w與t加總
                if not fold_in:
                    topic_word[ti, wi] += likelihood  # (道理同上)
                    topic_word_sum[ti] += likelihood
                    
        # Normalization
        for di in range(doc_count):
            for ti in range(topic_num):
                doc_topic[di, ti] /= doc_topic_sum[di]
        
        if not fold_in:
            for ti in range(topic_num):
                for wi in range(vocab_size):
                    topic_word[ti, wi] /= topic_word_sum[ti]  
                
    #################################################################
    #  log-likelihood (如果沒有early stop，結束前會最後一次計算目標函式)
    #################################################################
    # 這邊的算法跟上面完全一樣
    log_like = 0
    for dwi in range(nnz):
        di, wi = doc_term_row[dwi], doc_term_col[dwi]
        joint_prob_sum = 0
        for ti in range(topic_num):
            joint_prob_sum += doc_topic[di, ti] * topic_word[ti, wi]
        log_like += doc_term_data[dwi] * np.log(joint_prob_sum)
        
    if verbose > -1:
        print("iter", it, "- loglike:", log_like)

In [6]:
def normalize(np_array, axis=-1):
    ''' Sum of np_array along axis would be normalized to 1. '''
    return np_array / np_array.sum(axis=-1, keepdims=True)

# Randomize parameters of PLSA for both training and fold-in
doc_topic_train = normalize(np.random.rand(doc_count, topic_num))   # P(T|d)
topic_word = normalize(np.random.rand(topic_num, vocab_size))       # P(w|T)
doc_topic_fi = normalize(np.random.rand(doc_count_fi, topic_num))   # P(T|d) for fold-in usage

In [7]:
%%time
plsa(doc_term.data, doc_term.row, doc_term.col, doc_topic_train, topic_word)

iter 1 - loglike: -58375064.32942662
iter 2 - loglike: -42666864.24795003
iter 3 - loglike: -42631811.03370443
iter 4 - loglike: -42589930.5870872
iter 5 - loglike: -42533036.304297484
iter 6 - loglike: -42453760.52602836
iter 7 - loglike: -42342052.38765294
iter 8 - loglike: -42179724.784964636
iter 9 - loglike: -41944540.87725897
iter 10 - loglike: -41624785.13707598
iter 11 - loglike: -41246424.79358651
iter 12 - loglike: -40866619.35683942
iter 13 - loglike: -40530929.18316254
iter 14 - loglike: -40253326.85668143
iter 15 - loglike: -40027914.00614231
iter 16 - loglike: -39843226.18056094
iter 17 - loglike: -39689618.53012294
iter 18 - loglike: -39560262.943060406
iter 19 - loglike: -39450171.26666427
iter 20 - loglike: -39355719.72334994
iter 21 - loglike: -39274092.893165074
iter 22 - loglike: -39203099.82021385
iter 23 - loglike: -39141107.35095314
iter 24 - loglike: -39086534.97153943
iter 25 - loglike: -39038100.994601846
iter 26 - loglike: -38994892.14180201
iter 27 - loglike

In [8]:
%%time
plsa(doc_term_fi.data, doc_term_fi.row, doc_term_fi.col, doc_topic_fi, topic_word, fold_in=True)

iter 1 - loglike: -3006588.0233709137
iter 2 - loglike: -2882334.6224012002
iter 3 - loglike: -2832016.2191949
iter 4 - loglike: -2808913.0285848337
iter 5 - loglike: -2797308.326754359
iter 6 - loglike: -2790956.5827583554
iter 7 - loglike: -2787201.909543563
iter 8 - loglike: -2784834.618677411
iter 9 - loglike: -2783262.733025315
iter 10 - loglike: -2782175.165419598
iter 11 - loglike: -2781397.0331342653
iter 12 - loglike: -2780824.6267577736
iter 13 - loglike: -2780393.9192611133
iter 14 - loglike: -2780063.882542419
--- Early stopping at iteration 15 ---
loglike: -2779807.17344418
CPU times: user 3.02 s, sys: 76 ms, total: 3.1 s
Wall time: 3.04 s


In [9]:
%%time
w_unigram = alpha * normalize(doc_term_fi.toarray())
w_PLSA = beta * np.dot(doc_topic_fi, topic_word)

w_BGLM ={}
with open('BGLM.txt', mode='r') as file:
    for line in file:
        key, value = line.strip().split()
        w_BGLM[key] = (1 - alpha - beta) * np.exp(np.float(value))

CPU times: user 7.54 s, sys: 8.47 s, total: 16 s
Wall time: 1.5 s


In [10]:
%%time
with open("submission.csv", mode='w') as submit_file:
    submit_file.write("Query,RetrievedDocuments\n")

    query_list = os.listdir(query_dir)
    for query_name in query_list:
        submit_file.write(query_name + ",")
        
        query_path = query_dir + query_name
        with open(query_path) as query_file:
            log_scores = np.zeros((doc_count_fi))
            for line in query_file:
                for word in line.rstrip().split()[:-1]:
                    if word in vocab_table:
                        wi = vocab_table[word]
                        log_scores += np.log(w_unigram[:, wi] + w_PLSA[:, wi] + w_BGLM[word])
                    else:
                        log_scores += np.log(w_BGLM[word])
                        
        ranked_doc_idx = np.argsort(log_scores)[::-1]
        for idx in ranked_doc_idx:
            doc_name = doc_list[idx]
            submit_file.write(" " + doc_name)
        submit_file.write("\n")

CPU times: user 36 ms, sys: 4 ms, total: 40 ms
Wall time: 39.3 ms
